# Computing Nearest Neighbors Model for Songwriter Dataset

In [1]:
from collections import Counter
import json
import pickle

import dask
import dask.dataframe as dd
from dask_ml.wrappers import Incremental

import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors, kneighbors_graph

## Table of Contents

1. Loading Data
2. Computing Nearest Neighbors
3. Instituting Voting

## 1. Loading Data

In [ ]:
print('hello')

In [3]:
ddf = dd.read_csv('../data/interim/scaled_ddf_201901516.csv')\
        .rename(columns = {'Unnamed: 0' : 'index'})\
        .set_index('index')

## 2. Computing Nearest Neighbors

In [4]:
nn = NearestNeighbors(n_neighbors=10,
                      metric = 'euclidean')

In [5]:
nn_incremental = Incremental(nn)

In [8]:
fit_ddf = nn_incremental.fit(ddf)

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0) does not.

In [ ]:
euclidean_distances, euclidean_indices = nn.kneighbors(df)

In [ ]:
euclidean_distances.tofile('../data/final_model_poc/distances_20190428.fid')
euclidean_indices.tofile('../data/final_model_poc/indices_20190428.fid')

In [ ]:
import os

os.getcwd()

In [ ]:
os.chdir('../data/')

In [ ]:
os.mkdir('final_model_poc')

In [ ]:
os.chdir('../notebooks')

In [ ]:
distances.tofile('../data/final_model_test/distances_20190424.fid')
indices.tofile('../data/final_model_test/indices_20190424.fid')

### 2a. Merging Observations with Indices and Distances

In [ ]:
euclidean_distances = np.fromfile('../data/final_model_test/distances_20190428.fid')
euclidean_indices = np.fromfile('../data/final_model_test/indices_20190428.fid')

In [ ]:
euclidean_indices.astype(np.int16)[:900]

In [ ]:
euclidean_distances.astype(np.int16)[:60]

In [ ]:
distances = np.fromfile('../data/final_model_test/distances_20190424.fid')
indices = np.fromfile('../data/final_model_test/indices_20190424.fid')

In [ ]:
distances.shape

In [ ]:
df_distances = pd.DataFrame(np.vstack(np.split(euclidean_distances, 462965)))
df_indices = pd.DataFrame(np.vstack(np.split(euclidean_indices, 462965)))

In [ ]:
df_distances.shape

In [ ]:
df_indices.shape

In [ ]:
X_distances = pd.concat([df[['track_id',
                             'Song Title',
                             'artist_name',
                             'Writer Name',
                             'WID']],
                         df_distances],
                        axis = 1)

In [ ]:
X_indices = pd.concat([df[['track_id',
                             'Song Title',
                             'artist_name',
                             'Writer Name',
                             'WID']],
                         df_indices],
                        axis = 1)

In [ ]:
X_distances.head()

In [ ]:
X_indices.head()

In [ ]:
X_indices.iloc[146]

### 3. Determing Voting Mechanism

In [ ]:
X_voters = X_indices[['WID',
                      0,
                      1,
                      2,
                      3,
                      4,
                      5,
                      6,
                      7,
                      8,
                      9]]

In [ ]:
X_voters.columns

In [ ]:
X_voters_dict = X_voters.to_dict(orient = 'index')

In [ ]:
X_voters_dict[0]

In [ ]:
index_wid_dct = df['WID'].to_dict()

In [ ]:
(355, 7280) in index_wid_dct.items()

In [ ]:
len(index_wid_dct)

In [ ]:
def index_to_wid_mapping(df):
    index_wid_dict = df['WID'].to_dict()
    just_voters_df = df.drop('WID', axis = 1)
    new_voters_dict = {i:[] for i in range(len(just_voters_df))}
    
    count = 0
    for entry in just_voters_df.itertuples():
        for item in entry[:]:
            if item in index_wid_dict:
                new_voters_dict[count].append(index_wid_dict[item])
        count += 1
    
    new_voters_df = pd.DataFrame.from_dict(new_voters_dict,
                                           orient= 'index',
                                           columns = [i for i in range(11)])
    
    new_voters_df = pd.concat([pd.DataFrame.from_dict(index_wid_dict,
                                                      orient = 'index',
                                                      columns = ['WID']),
                               new_voters_df], axis = 1)

    return new_voters_df

In [ ]:
X_voters_2 = index_to_wid_mapping(X_voters)

In [ ]:
df[df['WID'] == 35426]

In [ ]:
X_voters[X_voters['WID'] == 105444]

In [ ]:
X_voters_2[X_voters_2['WID'] == 105444]

In [ ]:
def count_vals(df):
    id_dict = {wid : Counter() for wid in df['WID'].unique()}
    for entry in df.itertuples():
            id_dict[entry[1]].update(entry[2:])
    return id_dict

In [ ]:
# with just index values

count_df = count_vals(X_voters)

In [ ]:
# with WID values instead of index values

count_df_2 = count_vals(X_voters_2)

In [ ]:
count_df[7280].most_common(10)

In [ ]:
count_df_2[1940].most_common(10)

In [ ]:
count_df_2.keys()

##### Saving Writer Count Dict to File

In [ ]:
with open('../data/final_model_test/writer_count_dict_20190504.pkl', 'wb') as f:
    pickle.dump(count_df_2, f)

#### 3a. Mapping Votes back Against Writers

In [ ]:
# Will need to update this function as I figure out the logic for this algo

def combine_writer_counts_and_info(count_dict, writer_df):
    
    top_12_dict = {wid : {'writer_name' : '',
                          'ipi' : '',
                          'top_matches' : ''}\
                   for wid in count_dict.keys()}
    
    # Retreive Top 12 Results for each Songwriter
    for entry in count_dict:
        top_12_dict[entry]['top_matches'] = count_dict[entry].most_common(12)
    
    # add writer name and info to dictionary
    for wid in top_12_dict:
        if wid in writer_df['WID']:
            top_12_dict[wid]['writer_name']\
            = writer_df['Writer Name'][writer_df['WID'] == wid].values[0]
            top_12_dict[wid]['ipi']\
            = writer_df['IPI'][writer_df['WID'] == wid].values[0]
    
    return top_12_dict

In [ ]:
top_12s = combine_writer_counts_and_info(count_df_2, writers)

In [ ]:
top_12s[4692]

In [ ]:
writers = pd.read_csv('../data/writers.csv', index_col = 0)